# EM

In [1]:
import breeze.linalg._
import java.util.concurrent.ThreadLocalRandom
import breeze.stats.distributions._
import breeze.numerics._
val r=ThreadLocalRandom.current

val mu1 = DenseVector(1.0,2.0)
val sig1 = DenseMatrix((2.0,1.0),(1.0,2.0))
val mu2 = DenseVector(5.0,6.0)
val sig2 = DenseMatrix((4.0,1.0),(1.0,4.0))
val Gauss1 = new MultivariateGaussian(mu1,sig1)
val Gauss2 = new MultivariateGaussian(mu2,sig2)
val data = (0 until 10000).toArray.map(s => {
    if(r.nextDouble<0.4){
        Gauss1.draw  //directly draw! 
    }
    else {
        Gauss2.draw
    }
})

var Diff = 1.0
val paraData = sc.parallelize(data,4)
var estMu1 = paraData.reduce(_+_)/10000.0
var estMu2 = copy(estMu1) + 1.0
var estSig1 = paraData.map(s => (s - estMu1) * (s - estMu1).t).reduce(_+_)/10000.0
var estSig2 = copy(estSig1) + 1.0
val N = 10000.0
var Nk = 5000.0 
do{
    var OldMu1 = copy(estMu1)
    var OldMu2 = copy(estMu2)
    var OldSig1 = copy(estSig1)
    var OldSig2 = copy(estSig2)
    val Sifficient = paraData.map(x => {
        val x1 = - (x - estMu1).t * inv(estSig1) * (x - estMu1)/2
        val x2 = - (x - estMu2).t * inv(estSig2) * (x - estMu2)/2
        val gamma = (Nk * sqrt(det(estSig2))) / (Nk * sqrt(det(estSig2)) + (N - Nk) * sqrt(det(estSig1)) * exp(x2 - x1))
        (x,gamma,x * gamma,x * x.t * gamma,1.0 - gamma,(1.0 - gamma) * x,(1.0 - gamma) * x * x.t)
    })
    val Results = Sifficient.reduce((x,y) => (x._1 + y._1,x._2 + y._2,x._3 + y._3,x._4 + y._4,x._5 + y._5,x._6 + y._6,x._7 + y._7))
    Nk = Results._2
    estMu1 = Results._3/Nk
    estSig1 = Results._4/Nk - (estMu1 * estMu1.t)
    estMu2 = Results._6/Results._5
    estSig2 = Results._7/Results._5 - (estMu2 * estMu2.t)
    Diff = norm(estMu1 - OldMu1) + norm(estMu2 - OldMu2) + norm((estSig1 - OldSig1).toDenseVector) + norm((estSig2 - OldSig2).toDenseVector)
}while(Diff > 1e-6)

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1676778075546)
SparkSession available as 'spark'


23/02/19 11:41:26 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped
23/02/19 11:41:35 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/02/19 11:41:35 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
23/02/19 11:41:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/19 11:41:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


import breeze.linalg._
import java.util.concurrent.ThreadLocalRandom
import breeze.stats.distributions._
import breeze.numerics._
r: java.util.concurrent.ThreadLocalRandom = java.util.concurrent.ThreadLocalRandom@34d981a6
mu1: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0)
sig1: breeze.linalg.DenseMatrix[Double] =
2.0  1.0
1.0  2.0
mu2: breeze.linalg.DenseVector[Double] = DenseVector(5.0, 6.0)
sig2: breeze.linalg.DenseMatrix[Double] =
4.0  1.0
1.0  4.0
Gauss1: breeze.stats.distributions.MultivariateGaussian =
MultivariateGaussian(DenseVector(1.0, 2.0),2.0  1.0
1.0  2.0  )
Gauss2: breeze.stats.distributions.MultivariateGaussian =
MultivariateGaussian(DenseVector(5.0, 6.0),4.0  1.0
1.0  4.0  )
data: Array[breeze.linalg.DenseVector[Double]] = Array(DenseVector...


In [2]:
estMu1

res1: breeze.linalg.DenseVector[Double] = DenseVector(1.0177454758179587, 2.0346568202377666)


In [4]:
mu1

res3: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0)


# Lasso

In [1]:
import breeze.linalg._
import java.util.concurrent.ThreadLocalRandom
import breeze.stats.distributions._
import breeze.numerics._
val r=ThreadLocalRandom.current
val N = 1000
val p = 200
val beta = DenseVector.zeros[Double](p)
beta(0 to 2) := 2.0
val myGauss = new Gaussian(0,1)
val data = (0 until p).toArray.map(s => {
    val tmp = DenseVector(myGauss.sample(N).toArray)
    (s,tmp)
})

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1676788363138)
SparkSession available as 'spark'


23/02/19 14:32:55 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


import breeze.linalg._
import java.util.concurrent.ThreadLocalRandom
import breeze.stats.distributions._
import breeze.numerics._
r: java.util.concurrent.ThreadLocalRandom = java.util.concurrent.ThreadLocalRandom@6e632d29
N: Int = 1000
p: Int = 200
beta: breeze.linalg.DenseVector[Double] = DenseVector(2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....


In [31]:
val X = new DenseMatrix(rows = N,cols = p,data.map(s => s._2.toArray).foldLeft(Array(0.0))((x,y) => x.union(y)).drop(1))
//都是按列填充的啊！
//乱序的RDD中先打标签再抠出数据比较好，如上
//还有一种思路是直接先生成X和y再给他们打标签（不难吧？）
val y = X * beta + 0.2 * myGauss.draw
var estBeta = inv(X.t * X) * X.t * y
var OldBeta = copy(estBeta)

X: breeze.linalg.DenseMatrix[Double] =
-0.03248956493995189   -0.1135908116980738    ... (200 total)
-1.1101023293831114    1.2945385092190764     ...
-0.5059472624329526    -0.5110214509443586    ...
-0.1993887708619674    -0.48519139769063874   ...
-0.15626528275365892   -0.14656243001511596   ...
-1.1793457069562827    0.620101823606608      ...
-0.7136593089604237    0.5731009819563804     ...
1.1756983328102983     -1.4639528680423073    ...
0.16094586753064152    0.20234957028625528    ...
-0.7251365392850514    0.7064428704793227     ...
0.732119429229696      -0.4593231771780792    ...
-1.067005869191966     0.09107656742910852    ...
-0.34802920336598975   0.6348137715828132     ...
-0.6338737465506113    -0.336274212133304     ...
0.512253474712311      1.642498...


In [32]:
var Diff = 1.0
val lambda = 0.85e-12
val maxIter = 6
var ii = 0
while(Diff > 1e-6 & ii < maxIter){
    OldBeta := estBeta
    val tmp = data.map(s => {
        if(OldBeta(s._1) != 0.0){
            if(abs(s._2.t * (y - X * estBeta)) < lambda){
                0.0
            }
            else{
                s._2.t * (y - X * estBeta + s._2 * estBeta(s._1))/(lambda/abs(estBeta(s._1)) + s._2.t * s._2)
            }
        }else 0.0
    })
    estBeta = copy(DenseVector(tmp))
    ii += 1
    Diff = norm(OldBeta - estBeta)
    
}

Diff: Double = 3.392966230636183E-7
lambda: Double = 8.5E-13
maxIter: Int = 6
ii: Int = 6


In [33]:
estBeta

res27: breeze.linalg.DenseVector[Double] = DenseVector(2.003295145022895, 1.9979988243968831, 2.0075123003673814, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.005118273128754708, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...


In [35]:
new DenseMatrix(rows = N,cols = p,myGauss.sample(N*p).toArray)

res29: breeze.linalg.DenseMatrix[Double] =
1.4096011793308398     0.29419789506361005   ... (200 total)
1.183731765935585      0.08155584442066642   ...
0.79550709111341       -0.343239274701666    ...
-0.3019938854707638    0.3417109817488024    ...
0.5827090502413845     -0.7928479889345488   ...
1.6230950811121572     -0.5250570060431082   ...
0.975575167050472      -1.3132378764056518   ...
-0.180166212881233     -0.2303241798463296   ...
1.7699016656582325     -0.684788215649944    ...
0.5889220656793581     -0.3014593004437659   ...
0.7912458612276047     0.9292899361771099    ...
-1.0233483060943254    1.1479429091136804    ...
-0.2994557786678466    0.7453006275421138    ...
0.3182775487552813     0.514312980363929     ...
0.6201634353540946     1.6214080500360737...


In [1]:
import breeze.linalg._

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1677236446556)
SparkSession available as 'spark'


import breeze.linalg._


In [2]:
val testdense=new DenseMatrix(rows=2,cols=2,Array(1,2,3,4))

testdense: breeze.linalg.DenseMatrix[Int] =
1  3
2  4


In [4]:
testdense.rows

res1: Int = 2


In [6]:
val a=DenseVector(1.0,2.0,3.0,4.0)
DenseMatrix.tabulate(4,4){(i,j)=> a(i)*a(j)}

a: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0)
res3: breeze.linalg.DenseMatrix[Double] =
1.0  2.0  3.0   4.0
2.0  4.0  6.0   8.0
3.0  6.0  9.0   12.0
4.0  8.0  12.0  16.0


In [7]:
new Array[(Int,Int)](4)

res4: Array[(Int, Int)] = Array(null, null, null, null)


In [8]:
Array.ofDim[(Int,Double)](5)

res5: Array[(Int, Double)] = Array(null, null, null, null, null)


In [12]:
argmax(testdense(::,*))

res9: breeze.linalg.Transpose[breeze.linalg.DenseVector[Int]] = Transpose(DenseVector(1, 1))


In [14]:
import breeze.numerics._
log(DenseVector(1,2,3))

import breeze.numerics._
res11: breeze.linalg.DenseVector[Double] = DenseVector(0.0, 0.6931471805599453, 1.0986122886681098)


In [19]:
breeze.stats.meanAndVariance(Array(1.0,2.0,3.0))

res16: breeze.stats.meanAndVariance.MeanAndVariance = MeanAndVariance(2.0,1.0,3)


In [24]:
import org.apache.spark.mllib.random.RandomRDDs._
poissonRDD(sc,1,10000,10)

import org.apache.spark.mllib.random.RandomRDDs._
res21: org.apache.spark.rdd.RDD[Double] = RandomRDD[0] at RDD at RandomRDD.scala:42


In [25]:
Array(1,2,3).drop(1)

res22: Array[Int] = Array(2, 3)


In [30]:
import java.util.concurrent.ThreadLocalRandom
val r=ThreadLocalRandom.current
val mysample=(1 to 10000).toArray.map(s => {
    val tmp=r.nextDouble
    if(tmp<0.5){
        1
    } else -1
})
val mydouble=for(x <- mysample) yield x.toDouble

import java.util.concurrent.ThreadLocalRandom
r: java.util.concurrent.ThreadLocalRandom = java.util.concurrent.ThreadLocalRandom@41a09db8
mysample: Array[Int] = Array(-1, 1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, -1,...


In [31]:
breeze.stats.meanAndVariance(mydouble)

res28: breeze.stats.meanAndVariance.MeanAndVariance = MeanAndVariance(1.9999999999999524E-4,1.000099969996998,10000)


In [33]:
import breeze.stats.distributions._
val mygauss=new Gaussian(0,1)
mygauss.sample(4).toArray

import breeze.stats.distributions._
mygauss: breeze.stats.distributions.Gaussian = Gaussian(0.0, 1.0)
res30: Array[Double] = Array(-0.6304568590968392, 0.731062171414722, -0.009046777446450908, 0.551312351286247)


In [34]:
(1 to 100).map(s=> r.nextDouble) //IndexedSeq

res31: scala.collection.immutable.IndexedSeq[Double] = Vector(0.03189777723482212, 0.46642927924029787, 0.970445120504556, 0.30425960180545386, 0.4237223141301849, 0.057098033463492626, 0.8599653026604203, 0.4017773274721317, 2.849112386571484E-4, 0.056020950940620984, 0.8917874228205874, 0.35844989412861405, 0.40856070900431185, 0.9538644128376916, 0.3773165071761154, 0.04191952579233571, 0.7111976899540332, 0.6670412459101777, 0.5061788824279547, 0.7503814807815378, 0.40748685039740096, 0.3277018094312919, 0.41362361956978666, 0.27578562425226294, 0.5737170268233214, 0.4631865408872652, 0.8953105364651293, 0.21783627211879908, 0.2887047286128517, 0.9970735140077054, 0.12258054576055966, 0.5135612985272173, 0.5037363319370766, 0.6367514299412144, 0.4304051634493702, 0.4382995070293122,...


In [41]:
val mydir=new Dirichlet(DenseVector(3.0,2.0,6.0))
mydir.sample(5).toIndexedSeq

mydir: breeze.stats.distributions.Dirichlet[breeze.linalg.DenseVector[Double],Int] = Dirichlet(DenseVector(3.0, 2.0, 6.0))
res38: scala.collection.immutable.IndexedSeq[breeze.linalg.DenseVector[Double]] = Vector(DenseVector(0.3781676484971687, 0.09081445526622561, 0.5310178962366058), DenseVector(0.08027931589268758, 0.180593385361677, 0.7391272987456354), DenseVector(0.27691922375884426, 0.30273238508199335, 0.4203483911591624), DenseVector(0.33523665120538754, 0.2726473154594832, 0.39211603333512934), DenseVector(0.43098662885734546, 0.21714759553560625, 0.35186577560704835))


In [42]:
mydir.sample(5)

res39: IndexedSeq[breeze.linalg.DenseVector[Double]] = Vector(DenseVector(0.29068066872142556, 0.08126556559566804, 0.6280537656829064), DenseVector(0.41015210722766837, 0.08228499227825366, 0.507562900494078), DenseVector(0.46820181172002556, 0.07586780601091038, 0.4559303822690641), DenseVector(0.18183596203565058, 0.26488724959092846, 0.553276788373421), DenseVector(0.10978112363466846, 0.09135656564720067, 0.7988623107181309))


In [47]:
val mystr="1,1 2 3,2 3 4,3 4 5"
mystr.split(",").drop(1).map(t => t.split(" ").map(_.toDouble)).foldLeft(Array(0.0)){(x,y)=>x.union(y)}.drop(1)

mystr: String = 1,1 2 3,2 3 4,3 4 5
res44: Array[Double] = Array(1.0, 2.0, 3.0, 2.0, 3.0, 4.0, 3.0, 4.0, 5.0)


In [48]:
Array(1,2,3)(2)

res45: Int = 3


In [50]:
val xtmp=Array.fill[Double](4)(r.nextGaussian)
val y=DenseVector(xtmp).t*DenseVector(1.0,2.0,3.0,4.0)+r.nextGaussian

xtmp: Array[Double] = Array(0.3157882279855736, 0.6212197577762589, -0.5536511581143947, -0.024211027458427018)
y: Double = 0.682895518489464


In [53]:
val testrdd=sc.parallelize(Array(Array(1.0,2.0),Array(2.0,3.0)))
testrdd.map(s=> s(0)).collect()

testrdd: org.apache.spark.rdd.RDD[Array[Double]] = ParallelCollectionRDD[3] at parallelize at <console>:46
res47: Array[Double] = Array(1.0, 2.0)


In [55]:
val testKVP=sc.parallelize(Array((1,2.0),(1,3.0),(2,3.0),(2,5.0)))
testKVP.mapValues(s=> 1.0)

testKVP: org.apache.spark.rdd.RDD[(Int, Double)] = ParallelCollectionRDD[5] at parallelize at <console>:45
res49: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[6] at mapValues at <console>:48


In [56]:
val testmat=DenseMatrix((1.0,2.0,3.0),(2.0,3.0,4.0))
testmat(1,::)

testmat: breeze.linalg.DenseMatrix[Double] =
1.0  2.0  3.0
2.0  3.0  4.0
res50: breeze.linalg.Transpose[breeze.linalg.DenseVector[Double]] = Transpose(DenseVector(2.0, 3.0, 4.0))


In [1]:
val c = { val a = 5.0; val b=4}

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1677292136450)
SparkSession available as 'spark'


c: Unit = ()


23/02/25 10:29:11 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


In [2]:
Array.fill(4)(2.0)

res0: Array[Double] = Array(2.0, 2.0, 2.0, 2.0)


In [4]:
import breeze.linalg._
linspace(1,3,4)

import breeze.linalg._
res2: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 1.6666666666666665, 2.333333333333333, 3.0)


In [16]:
DenseMatrix((1.0,2.0),(2.0,3.0))*:*(DenseMatrix.ones[Double](2,2)*2.0)

res14: breeze.linalg.DenseMatrix[Double] =
2.0  4.0
4.0  6.0


In [12]:
DenseMatrix.ones[Double](2,2)*2.0

res10: breeze.linalg.DenseMatrix[Double] =
2.0  2.0
2.0  2.0


In [17]:
val a=sc.broadcast(1.0)
a.value

a: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(0)
res15: Double = 1.0


In [18]:
Array(1.0,2.0,3.0).map("%.4f" format _)

res16: Array[String] = Array(1.0000, 2.0000, 3.0000)


In [20]:
var OldEstMu1=DenseVector(0.0,0.0)
var EstMu1=OldEstMu1
OldEstMu1(0)=5.0

OldEstMu1: breeze.linalg.DenseVector[Double] = DenseVector(5.0, 0.0)
EstMu1: breeze.linalg.DenseVector[Double] = DenseVector(5.0, 0.0)


In [21]:
DenseVector(1.0,2.0).toArray

res19: Array[Double] = Array(1.0, 2.0)


In [22]:
DenseMatrix((1.0,2.0),(2.0,3.0)).toArray

res20: Array[Double] = Array(1.0, 2.0, 2.0, 3.0)
